<a href="https://colab.research.google.com/github/Jeferson-Peter/inSpeechSegmenter-AudioAnalysis/blob/master/AudioAnalysis.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

# Audio Analysis


## Instalando bibliotecas

In [ ]:
!pip install ffmpeg-python
!sudo apt-get install -y ffmpeg 
!pip install pydub
!pip install inaSpeechSegmenter
!pip install pyAudioAnalysis
!pip install eyed3
!pip install hmmlearn
!pip install SpeechRecognition

## Importando bibliotecas

In [2]:
import numpy as np
import IPython
from inaSpeechSegmenter import Segmenter
from inaSpeechSegmenter.export_funcs import seg2csv, seg2textgrid
from pydub import AudioSegment
import speech_recognition as sr
import sys

In [3]:
from google.colab import drive
drive.mount('/content/drive')

Drive already mounted at /content/drive; to attempt to forcibly remount, call drive.mount("/content/drive", force_remount=True).


In [4]:
import os
print(os.path)

## Tratamento do aúdio

In [5]:
IPython.display.display(IPython.display.Audio("18copy.wav"))

In [6]:
input_media = '18copy.wav'
output_media = '18copy_cleaned.wav'
output_path = 'audios_extracted'

Buscando momentos de silência e ruído

In [7]:
seg = Segmenter()
segmentation = seg(input_media)

6040200/6040200 [==============================] - 0s 0us/step


/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:86: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/pyannote/algorithms/utils/viterbi.py:95: FutureWarning: arrays to stack must be passed as a "sequence" type such as list or tuple. Support for non-sequence iterables such as generators is deprecated as of NumPy 1.16 and will raise an error in the future.
  return np.vstack(
/usr/local/lib/python3.8/dist-packages/inaSpeechSegmenter/segmenter.py:61: RuntimeWarning: invalid value encountered in subtract
  data = (data - np.mean(data, axis=1).reshape((len(data), 1))) / np.std(data, axis=1).reshape((len(data), 1))
/usr/local/lib/python3.8/dist-packages/numpy/core/_methods.py:230: RuntimeWarning: invalid value encountered in subtract
  x = asany

719/719 [==============================] - 148s 206ms/step


In [8]:
print(segmentation)

[('female', 0.0, 1.46), ('noEnergy', 1.46, 2.56), ('male', 2.56, 49.9), ('noise', 49.9, 70.62), ('male', 70.62, 84.16), ('noise', 84.16, 90.66), ('male', 90.66, 104.68), ('noise', 104.68, 107.4), ('male', 107.4, 109.92), ('noise', 109.92, 137.8), ('male', 137.8, 160.44), ('noise', 160.44, 164.12), ('male', 164.12, 168.64000000000001), ('noise', 168.64000000000001, 190.44), ('male', 190.44, 240.08), ('noise', 240.08, 241.82), ('male', 241.82, 252.48000000000002), ('noise', 252.48000000000002, 255.66), ('male', 255.66, 270.42), ('noise', 270.42, 294.64), ('male', 294.64, 333.7), ('noise', 333.7, 335.76), ('male', 335.76, 338.28000000000003), ('noise', 338.28000000000003, 340.86), ('male', 340.86, 348.3), ('noise', 348.3, 360.74), ('male', 360.74, 376.96), ('noise', 376.96, 379.76), ('male', 379.76, 381.16), ('noise', 381.16, 386.62), ('male', 386.62, 433.54), ('noise', 433.54, 438.16), ('male', 438.16, 485.02), ('noise', 485.02, 488.56), ('male', 488.56, 494.02000000000004), ('noise', 49

Gerando um csv destes dados

In [9]:
seg2csv(segmentation, 'myseg.csv')
# Here is the resulting CSV
# !cat myseg.csv

Realizando um filtro, para eliminar momentos de silêncio e ruído

In [10]:
import pandas as pd
df = pd.read_csv('myseg.csv', sep='\t')
df = df[(df['labels'] != 'noEnergy' ) & (df['labels'] != 'noise')]
df = df.reset_index(drop=True)
display(df)

,labels,start,stop
0,female,0.00,1.46
1,male,2.56,49.90
2,male,70.62,84.16
3,male,90.66,104.68
4,male,107.40,109.92
5,male,137.80,160.44
6,male,164.12,168.64
7,male,190.44,240.08
8,male,241.82,252.48
9,male,255.66,270.42


Criar aúdios a partir dos intervalos de tempo filtrados

In [11]:
from pydub import AudioSegment
import os
def cut_audio_from_wav(input_path, file_name, start, end, output_path, output_file):
  if not os.path.exists(output_path):
    os.makedirs(output_path)

  startSec = start * 1000
  endSec = end * 1000
  # Opening file and extracting segment
  song = AudioSegment.from_wav(input_path + file_name)
  extract = song[startSec:endSec]
  # Saving
  extract.export(f"{output_path}/{output_file}", format="wav")

with open('extract_audios.txt', 'w', encoding='utf-8') as  arq:
  total_lines = df.shape[0] -1
  for index, row in df.iterrows():
    try:
      
      output_file = f"_18copy_{index}.wav"
      cut_audio_from_wav('', '18copy.wav', row['start'], row['stop'],output_path, output_file)
      if index == total_lines:
        arq.write(f"file '{output_path}/{output_file}'")
      else:
        arq.write(f"file '{output_path}/{output_file}'\n")
      print(f'output_file: {output_file} created')
    except Exception as e:
      print(e)



output_file: _18copy_0.wav created
output_file: _18copy_1.wav created
output_file: _18copy_2.wav created
output_file: _18copy_3.wav created
output_file: _18copy_4.wav created
output_file: _18copy_5.wav created
output_file: _18copy_6.wav created
output_file: _18copy_7.wav created
output_file: _18copy_8.wav created
output_file: _18copy_9.wav created
output_file: _18copy_10.wav created
output_file: _18copy_11.wav created
output_file: _18copy_12.wav created
output_file: _18copy_13.wav created
output_file: _18copy_14.wav created
output_file: _18copy_15.wav created
output_file: _18copy_16.wav created
output_file: _18copy_17.wav created
output_file: _18copy_18.wav created
output_file: _18copy_19.wav created
output_file: _18copy_20.wav created
output_file: _18copy_21.wav created
output_file: _18copy_22.wav created


Leitura do arquivo txt gerado, ele irá concatenar todos os aúdios filtrados

In [12]:
import subprocess
txt_full_directory='extract_audios.txt'

# cmdargs = "ffmpeg -f concat -safe 0 -i " + txt_full_directory + " -c copy " + f"{output_media}"
cmd_args = f"ffmpeg -f concat -safe 0 -i {txt_full_directory} -c copy {output_media}"
# subprocess.run() will allow the process to end before creating a new one
try:
  subprocess.run(cmd_args, check=True, shell=True)
  print(f"File {output_media} successfully created")
except Exception as e:
  print(e)

File 18copy_cleaned.wav successfully created


## Transcrição do Aúdio


In [13]:
r = sr.Recognizer()
with sr.AudioFile(output_media) as source:
   audio_text = r.record(source)
   print(audio_text)
   text = r.recognize_google(audio_text,language='pt-BR')

result2:
{   'alternative': [   {   'confidence': 0.87411845,
                           'transcript': 'atendimento ficar boa tarde Gabriel '
                                         'com quem eu falo tudo bom você fala '
                                         'com as meninas tudo bem Como é que '
                                         'eu faço para te ajudando hoje'},
                       {   'transcript': 'atendimento ficar boa tarde Gabriel '
                                         'com quem eu falo tudo bom você fala '
                                         'com as meninas tudo bem Como é que '
                                         'eu faço para ter dado hoje'},
                       {   'transcript': 'atendimento ficar boa tarde Gabriela '
                                         'com quem eu falo tudo bom você fala '
                                         'com as meninas tudo bem Como é que '
                                         'eu faço para te ajudando hoje'}

In [14]:
text

'atendimento ficar boa tarde Gabriel com quem eu falo tudo bom você fala com as meninas tudo bem Como é que eu faço para te ajudando hoje'

In [15]:
with open(f'{output_media}_trancribed.txt', 'w') as arq:
  arq.write(text)